In [2]:
# import libraries
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.benchmarks.classic import SplitMNIST
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, \
    loss_metrics, timing_metrics, cpu_usage_metrics, confusion_matrix_metrics, disk_usage_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training.strategies import Naive
from avalanche.benchmarks.classic import CORe50, SplitTinyImageNet, SplitCIFAR10, SplitCIFAR100, SplitCIFAR110, SplitMNIST, RotatedMNIST, PermutedMNIST, SplitCUB200


import torch
import torch.nn as nn
import torch.nn.functional as F

from avalanche.models.dynamic_modules import MultiTaskModule, \
    MultiHeadClassifier
from avalanche.models.base_model import BaseModel

In [3]:
torch.__version__

'1.9.1'

In [2]:
# build models

class NetFC2x256(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self._input_size = 28*28
        self.fc1 = nn.Linear(28*28*1,256,bias=False)
        self.fc2 = nn.Linear(256, 256,bias=False)
        self.fc3 = nn.Linear(256, num_classes,bias=False)
        
        torch.nn.init.kaiming_uniform_(self.fc1.weight)
        torch.nn.init.kaiming_uniform_(self.fc2.weight)
        torch.nn.init.kaiming_uniform_(self.fc3.weight)

    def forward(self, x):
        #x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = x.contiguous()
        x = x.view(x.size(0), self._input_size)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    
class NetFC2x256grapes(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self._input_size = 28*28
        self.fc1 = nn.Linear(28*28*1,256,bias=False)
        self.fc2 = nn.Linear(256, 256,bias=False)
        self.fc3 = nn.Linear(256, num_classes,bias=False)
        
        torch.nn.init.kaiming_uniform_(self.fc1.weight)
        torch.nn.init.kaiming_uniform_(self.fc2.weight)
        torch.nn.init.kaiming_uniform_(self.fc3.weight)

    def forward(self, x):
        #x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = x.contiguous()
        x = x.view(x.size(0), self._input_size)
        x = F.relu(self.fc1(x))
        x = grapes_fc(x, self.fc1.weight)
        x = F.relu(self.fc2(x))
        x = grapes_fc(x, self.fc2.weight)
        x = self.fc3(x)
        return x
    
def grapes_fc(x,weights):
    correction_factor = torch.clip(2 * torch.sum(torch.abs(weights)) / torch.max (torch.sum(torch.abs(weights))) , 1,2)
    correction_factor = correction_factor.detach()
    return x * correction_factor - x.detach() * correction_factor + x.detach()


### Permutation

In [11]:
# build dataset
from avalanche.benchmarks.classic import CORe50, SplitTinyImageNet, SplitCIFAR10, SplitCIFAR100, SplitCIFAR110, SplitMNIST, RotatedMNIST, PermutedMNIST, SplitCUB200

# creating the benchmark (scenario object)
perm_mnist = PermutedMNIST(
    n_experiences=5,
    seed=8,    # seeds 1234,0,1,2,3
)

# recovering the train and test streams
train_stream = perm_mnist.train_stream
test_stream = perm_mnist.test_stream

# iterating over the train stream
for experience in train_stream:
    print("Start of task ", experience.task_label)
    print('Classes in this task:', experience.classes_in_this_experience)

    # The current Pytorch training set can be easily recovered through the 
    # experience
    current_training_set = experience.dataset
    # ...as well as the task_label
    print('Task {}'.format(experience.task_label))
    print('This task contains', len(current_training_set), 'training examples')

    # we can recover the corresponding test experience in the test stream
    current_test_set = test_stream[experience.current_experience].dataset
    print('This task contains', len(current_test_set), 'test examples')

Start of task  0
Classes in this task: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Task 0
This task contains 60000 training examples
This task contains 10000 test examples
Start of task  1
Classes in this task: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Task 1
This task contains 60000 training examples
This task contains 10000 test examples
Start of task  2
Classes in this task: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Task 2
This task contains 60000 training examples
This task contains 10000 test examples
Start of task  3
Classes in this task: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Task 3
This task contains 60000 training examples
This task contains 10000 test examples
Start of task  4
Classes in this task: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Task 4
This task contains 60000 training examples
This task contains 10000 test examples


In [12]:
model = NetFC2x256grapes(num_classes=perm_mnist.n_classes)

# log to Tensorboard
tb_logger = TensorboardLogger()

# log to text file
text_logger = TextLogger(open('log.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    forgetting_metrics(experience=True, stream=True),
    cpu_usage_metrics(experience=True),
    confusion_matrix_metrics(num_classes=perm_mnist.n_classes, save_image=False,
                             stream=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger, tb_logger]
)

# CREATE THE STRATEGY INSTANCE (NAIVE)
cl_strategy = Naive(
    model, SGD(model.parameters(), lr=0.001, momentum=0.9),
    CrossEntropyLoss(), train_mb_size=64, train_epochs=1, eval_mb_size=64,
    evaluator=eval_plugin)

# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in perm_mnist.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(perm_mnist.test_stream))

Starting experiment...
Start of experience:  0
Current Classes:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
-- >> Start of training phase << --
-- Starting training on experience 0 (Task 0) from train stream --
100%|████████████████████████████████████████████████████████████████████████████████| 938/938 [02:37<00:00,  5.94it/s]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 82264.6660
	DiskUsage_MB/train_phase/train_stream/Task000 = 82264.6660
	Loss_Epoch/train_phase/train_stream/Task000 = 0.3168
	Loss_MB/train_phase/train_stream/Task000 = 0.0967
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0002
	Time_Epoch/train_phase/train_stream/Task000 = 157.7642
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9083
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.9688
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|███████████████

	DiskUsage_Stream/eval_phase/test_stream/Task004 = 83068.8105
	Loss_Stream/eval_phase/test_stream/Task000 = 0.2478
	Loss_Stream/eval_phase/test_stream/Task001 = 0.1599
	Loss_Stream/eval_phase/test_stream/Task002 = 2.9975
	Loss_Stream/eval_phase/test_stream/Task003 = 3.6105
	Loss_Stream/eval_phase/test_stream/Task004 = 3.4970
	StreamForgetting/eval_phase/test_stream = 0.0284
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.9236
	Top1_Acc_Stream/eval_phase/test_stream/Task001 = 0.9527
	Top1_Acc_Stream/eval_phase/test_stream/Task002 = 0.1591
	Top1_Acc_Stream/eval_phase/test_stream/Task003 = 0.0500
	Top1_Acc_Stream/eval_phase/test_stream/Task004 = 0.0879
Start of experience:  2
Current Classes:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
-- >> Start of training phase << --
-- Starting training on experience 2 (Task 2) from train stream --
100%|████████████████████████████████████████████████████████████████████████████████| 938/938 [02:57<00:00,  5.27it/s]
Epoch 0 ended.
	DiskUsage_Epoch/train_phas

-- Starting eval on experience 4 (Task 4) from test stream --
100%|████████████████████████████████████████████████████████████████████████████████| 157/157 [00:26<00:00,  5.90it/s]
> Eval on experience 4 (Task 4) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task004/Exp004 = 215.5370
	DiskUsage_Exp/eval_phase/test_stream/Task004/Exp004 = 84373.5576
	Loss_Exp/eval_phase/test_stream/Task004/Exp004 = 3.7236
	Top1_Acc_Exp/eval_phase/test_stream/Task004/Exp004 = 0.0910
-- >> End of eval phase << --
	ConfusionMatrix_Stream/eval_phase/test_stream = 
tensor([[3680,  105,  328,  ...,    0,    0,    0],
        [   5, 4452, 1005,  ...,    0,    0,    0],
        [  74,   33, 3804,  ...,    0,    0,    0],
        ...,
        [   0,    0,    0,  ...,    0,    0,    0],
        [   0,    0,    0,  ...,    0,    0,    0],
        [   0,    0,    0,  ...,    0,    0,    0]])
	DiskUsage_Stream/eval_phase/test_stream/Task004 = 84373.5576
	Loss_Stream/eval_phase/test_stream/Task000 = 0